## Comparison Between Approximate P+M Solution and Strain Compatibility Analysis

See blog post here: https://robwang.io/engineering/2021/10/22/wallflexure.html


In [1]:
# Import
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import itertools
import os
import ipywidgets

In [2]:
# Constants
REBAR = {
    3:[0.375, 0.11],
    4:[0.500, 0.20],
    5:[0.625, 0.31],
    6:[0.750, 0.44],
    7:[0.875, 0.60],
    8:[1.000, 0.79],
    9:[1.128, 1.00],
    10:[1.270, 1.27],
    11:[1.410, 1.56],
    14:[1.693, 2.25],
    18:[2.257, 4.00]}

### Define Section class

In [3]:
class Section:
    """
    Attributes:
        f'c = concrete strength
        fy = steel strength
        b = wall thickness
        h = wall length
        cb = clear cover = 2 in
        nx = number of column of bars in boundary element = 3
        ny = number of rows of bars in boundary element
        BE_bar = boundary element bar size
        Lbe = boundary element length
        web_bar = web bar size
        s_target = target spacing in web
        n_curtain = number of curtains in web = 2

        P_max = maximum allowed axial load
        ext_pt = list of points of four corners of wall
        reinf_coord = rebar location coordinate
        PMsurface = coordinate for P+M surface
        PMsurface_approx = coordinate of P+M surface using approximate equation
    """
    def __init__(self,fpc,fy,b,h,cb,nx,ny,BE_bar,Lbe,web_bar,s_target,n_curtain,zeta):
        self.fpc = fpc
        self.fy = fy
        self.b = b
        self.h = h
        self.cb = cb
        self.nx = nx
        self.ny = ny
        self.BE_bar = BE_bar
        self.Lbe = Lbe
        self.web_bar = web_bar
        self.s_target = s_target
        self.n_curtain = n_curtain
        self.zeta = zeta
        self.ext_pt = self.get_ext_pts()
        self.BE_reinf,self.web_reinf,self.reinf_coord = self.get_reinf_coord()
        self.determine_PM()
        self.determine_approx_PM()
        self.factored_PM()
    

    def get_ext_pts(self):
        ext_pts=[[-self.b/2,self.b/2,self.b/2,-self.b/2,-self.b/2,],
        [-self.h/2,-self.h/2,self.h/2,self.h/2,-self.h/2,]]
        return ext_pts



    def draw_BE_bars(self,x,y,b,h,cb,nx,ny,A,D,layout):
        """
        This function generates the x,y coordinate of boundary element steel based on
        the following input:
        x : dx from origin to lower left of boundary element
        y : dy from origin to lower left of boundary element
        b : dimension of BE along x
        h : dimension of BE along y
        cb : clear cover to face of rebar
        nx : number of columns along x
        ny : number of columns along y
        A : area of rebar
        D : diameter of rebar
        layout : layout. f for "full" or p for "perimeter"
        """
        # first determine spacing and coordinate with local origin at lower left
        cx = b - 2*cb - D
        cy = h - 2*cb - D
        sx = 0 if nx==1 else cx / (nx-1)
        sy = 0 if ny==1 else cy / (ny-1)
            
        xcoord=[]
        ycoord=[]
        xcoord.append(cb+D/2)
        ycoord.append(cb+D/2)
        if sx!=0:
            for i in range(nx-1):
                xcoord.append(xcoord[-1]+sx)
        if sy!=0:
            for i in range(ny-1):
                ycoord.append(ycoord[-1]+sy)
        reinfarray_local = list(itertools.product(xcoord,ycoord))
        
        if layout == "p":
            x0 = cb+D/2
            xn = xcoord[-1]
            y0 = cb+D/2
            yn = ycoord[-1]
            reinfarray_local = [e for e in reinfarray_local if e[0]==x0 or e[0]==xn or e[1]==y0 or e[1]==yn]
                
        # translate local origin to global
        reinfarray_global=[]
        for i in range(len(reinfarray_local)):
            x_global = reinfarray_local[i][0] + x 
            y_global = reinfarray_local[i][1] + y 
            reinfarray_global.append([A,x_global,y_global])
            
        return reinfarray_global



    def draw_web_bars(self,x,y,b,h,cb,s_target,A,D,ncurtain,direction):
        """
        This function generates the x,y coordinate of boundary element steel based on
        the following input:
        x : dx from origin to lower left corner of web
        y : dy from origin to lower left corner of web
        b : dimension along x
        h : dimension along y
        cb : clear cover to face of rebar
        s_target : target spacing of bars
        A : area of rebar
        D : diameter of rebar
        ncurtain: number of curtains
        direction: vertical or horizontal
        """
        xcoord=[]
        ycoord=[]
        reinfarray_local=[]
        if direction == "v":
            if ncurtain==1:
                N_L = math.floor((h)/s_target)
                if N_L==0:
                    return []
                s_L = (h)/(N_L)
                xcoord.append(0)
                ycoord.append(s_L/2)
                for i in range(N_L-1):
                    ycoord.append(ycoord[-1]+s_L)
                reinfarray_local = list(itertools.product(xcoord,ycoord))
            else:
                s_t = (b-cb-cb-D)/(ncurtain-1)
                N_L = math.floor((h)/s_target)
                if N_L==0:
                    return []
                s_L = (h)/(N_L)
                xcoord.append(cb+D/2)
                ycoord.append(s_L/2)
                for i in range(ncurtain-1):
                    xcoord.append(xcoord[-1]+s_t)
                for i in range(N_L-1):
                    ycoord.append(ycoord[-1]+s_L)
                reinfarray_local = list(itertools.product(xcoord,ycoord))
        elif direction == "h":
            if ncurtain==1:
                N_L = math.floor((b)/s_target)
                if N_L==0:
                    return []
                s_L = (b) / (N_L)
                ycoord.append(0)
                xcoord.append(s_L/2)
                for i in range(N_L-1):
                    xcoord.append(xcoord[-1]+s_L)
                reinfarray_local = list(itertools.product(xcoord,ycoord))
            else:
                s_t = (h-cb-cb-D)/(ncurtain-1)
                N_L = math.floor((b)/s_target)
                if N_L==0:
                    return []
                s_L = (b) / (N_L)
                ycoord.append(cb+D/2)
                xcoord.append(s_L/2)
                for i in range(ncurtain-1):
                    ycoord.append(ycoord[-1]+s_t)
                for i in range(N_L-1):
                    xcoord.append(xcoord[-1]+s_L)
                reinfarray_local = list(itertools.product(xcoord,ycoord))
        # translate local origin to global
        reinfarray_global=[]
        for i in range(len(reinfarray_local)):
            x_global = reinfarray_local[i][0] + x 
            y_global = reinfarray_local[i][1] + y 
            reinfarray_global.append([A,x_global,y_global])
        return reinfarray_global



    def get_reinf_coord(self):
        """
        function to generate reinforcement coordinates
        """
        # Draw BE steel
        BE_reinf=[]
        dx = -self.b/2
        dy = -self.h/2
        A = REBAR[self.BE_bar][1]
        D = REBAR[self.BE_bar][0]
        BE_reinf.append(self.draw_BE_bars(dx,dy,self.b,self.Lbe,self.cb,self.nx,self.ny,A,D,"p"))
        dx = -self.b/2
        dy = self.h/2 - self.Lbe
        A = REBAR[self.BE_bar][1]
        D = REBAR[self.BE_bar][0]
        BE_reinf.append(self.draw_BE_bars(dx,dy,self.b,self.Lbe,self.cb,self.nx,self.ny,A,D,"p"))
        BE_reinf = BE_reinf[0] + BE_reinf[1]
        
        
        # Draw web steel
        dx = -self.b/2
        dy = -self.h/2 + self.Lbe
        h = self.h - 2*self.Lbe
        s = self.s_target
        A = REBAR[self.web_bar][1]
        D = REBAR[self.web_bar][0]
        web_reinf=self.draw_web_bars(dx,dy,self.b,h,self.cb,s,A,D,self.n_curtain,'v')

        # combine all reinf coordinate into one list
        reinfarray = BE_reinf + web_reinf
                
        return BE_reinf,web_reinf,reinfarray


        
    def PM(self,c):
        """
        Algorithm to determine PM surface as follows (From CRSI)
            0.) determine Pmax
            1.) assume c values from 0.01 to N where the last step is sufficiently close to Pmax
            2.) determine beta
            3.) determine a
            4.) calculate compression force resultant C
            5.) loop through every single reinforcement
                5a.) calculate rebar depth
                5b.) calculate rebar strain & stress
                5c.) calculate rebar force
            6.) calculate Pn and Mn of the section
            7.) back to step 1, assume another c value
        """
        # step 2
        if self.fpc < 4:
            beta = 0.85
        elif self.fpc > 8:
            beta = 0.65
        else:
            beta = 0.85 - 0.05*(self.fpc*1000-4000)/1000
        
        # step 3
        a = beta*c
        
        # step 4
        C = 0.85*self.fpc*a*self.b
        
        # step 5 (-ve is tensile strain)
        F_list=[]
        d_list=[]
        for bar in self.reinf_coord:
            # step 5a
            di = self.h/2 - bar[2]
            Asi = bar[0]
            
            # step 5b
            esi = 0.003*(c-di)/c
            fsi = 29000*esi
            if fsi<0 and fsi<-self.fy:
                fsi = -self.fy
            elif fsi>0 and fsi>self.fy:
                fsi = self.fy
                
            # step 5c
            if di > a:
                Fsi = fsi*Asi
            else:
                Fsi = (fsi-0.85*self.fpc)*Asi
            F_list.append(Fsi)
            d_list.append(di)
            
        # step 6
        P = C + sum(F_list)
        M = 0.5*C*(self.h-a) + sum([A*(0.5*self.h-B) for A,B in zip(F_list,d_list)])
        M = M/12
        
        # step 7 calculuate strain at extreme fiber and phi factor
        ey = self.fy / 29000
        d = self.h - self.cb
        et = (d-c)/c * 0.0031
        
        if et >ey+0.003:
            phi = 0.9
        elif et>ey and et<ey+0.003:
            phi = 0.75 + 0.15*(et-ey)/((ey+0.003)-ey)
        elif et<ey:
            phi = 0.65
        
        return P,M,c,phi




    def approx_PM(self,Pu,fy,lw,tw,fpc,lwb,lww,Asl1,Asl2):   
        """
        determine interaction surface with approximate equation
        Notation:
            Mn1 = moment contribution from BE
            Mn2 = moment contribution from web
            Fy = yield strength of rebar
            Pu = axial demand
            Aslb = area of steel in boundary element
            Aslw = area of steel in web
            Asl1 = area of equivalent steel plate
            Asl2 = area of boundary steel adjusted
            lw = length of wall
            lwb = length of BE
            lww = length of wall web
            tw = thickness of wall
            c = depth of neutral axis
            w = reinforcement ratio
            alpha = axial load ratio
            beta = stress block parameter
            
        1.) axial load ratio
                alpha = Pu/fpc/lw/tw
        2.) beta stress block
                beta = 0.85 - 0.05*(fpc-4000)/1000, between 0.65 and 0.85
        3.) reinforcement ratio
                w = Asl1*Fy/lw/tw/fpc
        4.) neutral axis depth
                c = (w + alpha) / (2*w+0.85*beta)
        5.) moment from boundary steel
                Mn2 = Asl2*Fy*(lw-lwb)
        6.) moment from web steel
                Mn1 = 0.5*Asl1*Fy*lw*(1+Pu/Asl1/Fy)*(1-c/lw)
        7.) total nominal moment capacity
                Mn = Mn1 + Mn2
        """
        ZETA=self.zeta
        # step 1
        Ag = self.b * self.h
        Abe = len(self.BE_reinf) * self.BE_reinf[0][0]
        Aweb = len(self.web_reinf) * self.web_reinf[0][0]
        Ast = Abe + Aweb
        Pmax = 0.85*self.fpc *(Ag - Ast) + self.fy * Ast
        
        alpha = Pu/(self.fpc*Ag)
        
        # step 2
        if fpc < 4:
            beta = 0.85
        elif fpc > 8:
            beta = 0.65
        else:
            beta = 0.85 - 0.05*(fpc*1000-4000)/1000
            
        # step 3
        w = Asl1*fy/lw/tw/fpc
        
        # step 4
        c = (w + alpha) / (2*w+0.85*beta)*lw
        
        # step 5
        Mn2 = Asl2*fy*(lw-lwb)
        
        # step 6
        Mn1 = 0.5*Asl1*fy*lw*(1+Pu/Asl1/fy)*(1-ZETA*c/lw)
        
        # step 7
        Mn = (Mn1 + Mn2)/12
        Pn = Pu
        
        # step 8 calculuate strain at extreme fiber and phi factor
        ey = fy / 29000
        d = self.h - self.cb
        et = (d-c)/c * 0.003
        if c<0:
            et = ey+0.0031
        
        if et >ey+0.003:
            phi = 0.9
        elif et>ey and et<ey+0.003:
            phi = 0.75 + 0.15*(et-ey)/(ey+0.003-ey)
        elif et<ey:
            phi = 0.65
            
        return Pn, Mn, c, phi
            
        

    def determine_approx_PM(self):
        """
        return a list of coordinates for the approximate PM surface
        """
        # constants
        fy = self.fy
        lw = self.h
        tw = self.b
        fpc = self.fpc
        lwb = self.Lbe
        lww = self.h - 2*self.Lbe
        Ag = lw*tw
        
        # determine steel quantity
        Aslb = sum([A[0] for A in self.BE_reinf])/2
        Aslw = sum([A[0] for A in self.web_reinf])
        Astotal = Aslb*2 + Aslw
        Asl1 = Aslw * lw/lww
        Asl2 = Aslb - Aslw * lwb / lww
              
        # generate PM points
        P_coord=[]
        M_coord=[]
        c_coord=[]
        phi_coord=[]
        P_list = np.linspace(self.Tmin,self.Pmax,100)
        for p in P_list:
            Pn,Mn,c,phi = self.approx_PM(p,fy,lw,tw,fpc,lwb,lww,Asl1,Asl2)
            P_coord.append(Pn)
            M_coord.append(Mn)
            c_coord.append(c)
            phi_coord.append(phi)
        
        self.PMsurface_approx = [P_coord,M_coord,c_coord,phi_coord]


    def determine_PM(self):
        """
        function used to determine accurate PM surface via strain compatibility
        """
        # determine steel and concrete areas
        Ag = self.b * self.h
        Abe = len(self.BE_reinf) * self.BE_reinf[0][0]
        Aweb = len(self.web_reinf) * self.web_reinf[0][0]
        Ast = Abe + Aweb
        self.Pmax = 0.85*self.fpc *(Ag - Ast) + self.fy * Ast
        self.Tmin = -Ast * self.fy
        
        # first step with c = 0.01. Full tension
        P_coord=[]
        M_coord=[]
        c_coord=[]
        phi_coord=[]
        cstep=0.01
        P,M,c,phi = self.PM(cstep)
        P_coord.append(P)
        M_coord.append(M)
        c_coord.append(c)
        phi_coord.append(phi)
        
        
        # loop and increment c by 1 inch every loop until reaching 0.8Po
        P=0
        cstep=1
        while abs(P-self.Pmax)>300:
            P,M,c,phi = self.PM(cstep)
            P_coord.append(P)
            M_coord.append(M)
            c_coord.append(c)
            phi_coord.append(phi)
            cstep = cstep +1
        
        self.PMsurface = [P_coord,M_coord,c_coord,phi_coord]
            
    
    def factored_PM(self):
        self.PMsurface_factored = [[],[],[]]
        self.PMsurface_approx_factored = [[],[],[]]
        
        for i in range(len(self.PMsurface[0])):
            if self.PMsurface[0][i] < self.Pmax*0.8:
                p=self.PMsurface[0][i] * self.PMsurface[3][i]
                m=self.PMsurface[1][i] * self.PMsurface[3][i]
                c=self.PMsurface[2][i]
                self.PMsurface_factored[0].append(p)
                self.PMsurface_factored[1].append(m)
                self.PMsurface_factored[2].append(c)
        
        for i in range(len(self.PMsurface_approx[0])):
            if self.PMsurface_approx[0][i] < self.Pmax*0.8:
                p = self.PMsurface_approx[0][i]* self.PMsurface_approx[3][i]
                m = self.PMsurface_approx[1][i]* self.PMsurface_approx[3][i]
                c = self.PMsurface_approx[2][i]
                self.PMsurface_approx_factored[0].append(p)
                self.PMsurface_approx_factored[1].append(m)
                self.PMsurface_approx_factored[2].append(c)
        
        # Add last point at 0.8Po
        self.PMsurface_factored[0].append(self.PMsurface_factored[0][-1])
        self.PMsurface_factored[1].append(0)
        self.PMsurface_factored[2].append(self.PMsurface_factored[2][-1])
        self.PMsurface_approx_factored[0].append(self.PMsurface_approx_factored[0][-1])
        self.PMsurface_approx_factored[1].append(0)
        self.PMsurface_approx_factored[2].append(self.PMsurface_approx_factored[2][-1])


    def plot(self):
        # initialize plot
        fig, axs = plt.subplots(1,3,figsize=(16,9),gridspec_kw={'width_ratios':[1,3,3]})
        
        # plot section
        axs[0].add_patch(patches.Rectangle([-self.b/2,-self.h/2], self.b, self.h, color = "grey" ))
        BE_x, BE_y, web_x, web_y=[],[],[],[]
        for coord in self.BE_reinf:
            BE_x.append(coord[1])
            BE_y.append(coord[2])
        for coord in self.web_reinf:
            web_x.append(coord[1])
            web_y.append(coord[2])
        #size of marker range from 2 to 6 for #4 to #11
        websize =0.5*self.web_bar + 0.5
        BEsize = 0.5*self.BE_bar + 0.5
        axs[0].plot(BE_x,BE_y,marker=".",markersize=BEsize,color="black",linestyle="None")
        axs[0].plot(web_x,web_y,marker=".",markersize=websize,color="black",linestyle="None")
        # update styling 
        axs[0].set_xlim([-self.h/2*1.05,self.h/2*1.05])
        axs[0].set_ylim([-self.h/2*1.05,self.h/2*1.05])
        axs[0].set_title("Section Layout - h = {:.1f} ft".format(self.h))
        
        
        # plot PM
        axs[1].plot(self.PMsurface_approx[1],self.PMsurface_approx[0],
                    color="deepskyblue",linestyle="dashed",label="Approximation")
        axs[1].plot(self.PMsurface[1],self.PMsurface[0],color="darkviolet",label="Strain Compatibility")
        # axis limits
        Mmax=max(self.PMsurface[1])
        Pmax=max(self.PMsurface[0])
        Pmin=min(self.PMsurface[0])
        axs[1].set_xlim([0,Mmax*1.05])
        #axs[1].set_ylim([Pmin*1.05,Pmax*1.05])
        #axs[1].set_xlim([-200000,200000])
        #axs[1].set_ylim([-5000,50000])
        # update styling
        axs[1].set_title("PM Surface Comparison")
        axs[1].xaxis.grid()
        axs[1].yaxis.grid()
        axs[1].axhline(0,color="black")
        axs[1].axvline(0,color="black")
        axs[1].set_ylabel("Axial (kips)")
        axs[1].set_xlabel("Moment (kips.ft)")
        axs[1].legend(loc="upper right")
        
        
        
        # plot P vs. c
        axs[2].plot(self.PMsurface_approx[2],self.PMsurface_approx[0],color="deepskyblue",linestyle="dashed",label="Approximation")
        axs[2].plot(self.PMsurface[2],self.PMsurface[0],color="darkviolet",label="Strain Compatibility")
        # update styling
        axs[2].set_title("P vs. c")
        axs[2].xaxis.grid()
        axs[2].yaxis.grid()
        axs[2].set_xlabel("Neutral Axis Depth (in)")
        axs[2].set_ylabel("Axial Load (kips)")
        axs[2].legend(loc="upper right")
        # axis limits
        Pmin=min(self.PMsurface[0])
        Pmax=max(self.PMsurface[0])
        cmax=max(self.PMsurface[2])
        axs[2].set_ylim([Pmin*1.05,Pmax*1.05])
        axs[2].set_xlim([0,cmax*1.05])
        axs[2].axhline(0,color="black")
        axs[2].axvline(0,color="black")
        
        return fig

    def plot_factored(self):
        # initialize plot
        fig, axs = plt.subplots(1,3,figsize=(16,9),gridspec_kw={'width_ratios':[1,3,3]})
        
        # plot section
        axs[0].add_patch(patches.Rectangle([-self.b/2,-self.h/2], self.b, self.h, color = "grey" ))
        BE_x, BE_y, web_x, web_y=[],[],[],[]
        for coord in self.BE_reinf:
            BE_x.append(coord[1])
            BE_y.append(coord[2])
        for coord in self.web_reinf:
            web_x.append(coord[1])
            web_y.append(coord[2])
        #size of marker range from 2 to 6 for #4 to #11
        websize =0.5*self.web_bar + 0.5
        BEsize = 0.5*self.BE_bar + 0.5
        axs[0].plot(BE_x,BE_y,marker=".",markersize=BEsize,color="black",linestyle="None")
        axs[0].plot(web_x,web_y,marker=".",markersize=websize,color="black",linestyle="None")
        # update styling 
        axs[0].set_xlim([-self.h/2*1.05,self.h/2*1.05])
        axs[0].set_ylim([-self.h/2*1.05,self.h/2*1.05])
        axs[0].set_title("Section Layout - h = {:.1f} ft".format(self.h))
        
        
        # plot PM
        axs[1].plot(self.PMsurface_approx_factored[1],self.PMsurface_approx_factored[0],
                    color="deepskyblue",linestyle="dashed",label="Approximation")
        axs[1].plot(self.PMsurface_factored[1],self.PMsurface_factored[0],color="darkviolet",label="Strain Compatibility")
        # axis limits
        Mmax=max(self.PMsurface_factored[1])
        Pmax=max(self.PMsurface_factored[0])
        Pmin=min(self.PMsurface_factored[0])
        axs[1].set_xlim([0,Mmax*1.05])
        #axs[1].set_ylim([Pmin*1.05,Pmax*1.05])
        #axs[1].set_xlim([-200000,200000])
        #axs[1].set_ylim([-5000,50000])
        # update styling
        axs[1].set_title("PM Surface Comparison")
        axs[1].xaxis.grid()
        axs[1].yaxis.grid()
        axs[1].axhline(0,color="black")
        axs[1].axvline(0,color="black")
        axs[1].set_ylabel("Axial (kips)")
        axs[1].set_xlabel("Moment (kips.ft)")
        axs[1].legend(loc="upper right")
        
        
        
        # plot P vs. c
        axs[2].plot(self.PMsurface_approx[2],self.PMsurface_approx[0],color="deepskyblue",linestyle="dashed",label="Approximation")
        axs[2].plot(self.PMsurface[2],self.PMsurface[0],color="darkviolet",label="Strain Compatibility")
        # update styling
        axs[2].set_title("P vs. c")
        axs[2].xaxis.grid()
        axs[2].yaxis.grid()
        axs[2].set_xlabel("Neutral Axis Depth (in)")
        axs[2].set_ylabel("Axial Load (kips)")
        axs[2].legend(loc="upper right")
        # axis limits
        Pmin=min(self.PMsurface[0])
        Pmax=max(self.PMsurface[0])
        cmax=max(self.PMsurface[2])
        axs[2].set_ylim([Pmin*1.05,Pmax*1.05])
        axs[2].set_xlim([0,cmax*1.05])
        axs[2].axhline(0,color="black")
        axs[2].axvline(0,color="black")
        
        return fig

## Interactive Plot
The following parameters may be changed by user:
* $f'_c$ = 4 ksi to 8 ksi
* $f_y$ = 60 ksi to 100 ksi
* $t$ = 12" to 36"
* $L$ = 60" to 500"
* $L_{BE}$ = 12" to 60"
* $A_{wbar}$ = \#3 to \#11
* $A_{bbar}$ = \#3 to \#11
* $s_{web}$ = 4" to 12"

The following parameters were set to a constant value as they do not affect the results much:
* clear cover will always be 2"
* boundary element will always be 3 layers
* boundary element vertical bar spacing will always be 4". Amount of steel changed by $L_{BE}$. Longer length equals more boundary steel


In [4]:
# default values
h=160
fpc=6
fy=60
b=24
BE_bar=10
web_bar=6
s_target=6
Lbe=24

In [5]:
def plot(fpc,fy,b,h,BE_bar,Lbe,web_bar,s_target,zeta,flag):
    nx=3
    n_curtain=2
    cb=2
    s_BE=4
    ny=round(Lbe/s_BE)
    my_section = Section(fpc,fy,b,h,cb,nx,ny,BE_bar,Lbe,web_bar,s_target,n_curtain,zeta)
    if flag =="nominal":
        PM_fig = my_section.plot()
    else:
        PM_fig = my_section.plot_factored()

In [6]:
slider1 = ipywidgets.IntSlider(value=6,min=4,max=8,step=1,description="f'c",
                               disabled=False,continuous_update=True, readout=True)
slider2 = ipywidgets.IntSlider(value=60,min=60,max=100,step=20,description="fy",
                               disabled=False,continuous_update=True, readout=True)
slider3 = ipywidgets.IntSlider(value=24,min=12,max=48,step=6,description="b",
                               disabled=False,continuous_update=True, readout=True)
slider4 = ipywidgets.IntSlider(value=160,min=60,max=500,step=20,description="L",
                               disabled=False,continuous_update=True, readout=True)
slider5 = ipywidgets.IntSlider(value=24,min=12,max=60,step=6,description="L_be",
                               disabled=False,continuous_update=True, readout=True)
slider6 = ipywidgets.IntSlider(value=6,min=4,max=12,step=2,description="spacing",
                               disabled=False,continuous_update=True, readout=True)
slider7 = ipywidgets.IntSlider(value=6,min=3,max=11,step=1,description="web bar",
                               disabled=False,continuous_update=True, readout=True)
slider8 = ipywidgets.IntSlider(value=10,min=3,max=11,step=1,description="boundary bar",
                               disabled=False,continuous_update=True, readout=True)
slider9 = ipywidgets.FloatSlider(value=0.83,min=0.6,max=1.5,step=0.01,description="zeta",
                               disabled=False,continuous_update=True, readout=True)
dropdown10 = ipywidgets.Dropdown(options=['factored','nominal'],value='nominal',description='Type',disabled=False)

In [7]:
ipywidgets.interact(plot,fpc=slider1,fy=slider2,b=slider3,h=slider4,
                    BE_bar=slider8,Lbe=slider5,web_bar=slider7,s_target=slider6,zeta=slider9,flag=dropdown10)

interactive(children=(IntSlider(value=6, description="f'c", max=8, min=4), IntSlider(value=60, description='fy…

<function __main__.plot(fpc, fy, b, h, BE_bar, Lbe, web_bar, s_target, zeta, flag)>